### Step 1: Set up the workspace

In [5]:
import json
from azureml.core import Workspace
from azureml.core.model import Model

#GETTING MY ID:
id = open('id_confidential_david.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

from azureml.core import Workspace

ws = Workspace.create(name="Class2",
                      subscription_id = my_id,
                      resource_group = "__hw1__",
                      location = "centralindia")

mname = "model"
registered_model = Model.register(model_path="random_forest_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

AttributeError: 'tuple' object has no attribute 'lower'

################ score.py ###################
scorepy = f"""
import json
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def run(raw_data):
    try: ## Try la predicción.
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)
        
        data_dummies = data.drop(selected_columns.json, axis=1)

        result = model.predict(data_dummies).tolist()
        return json.dumps(result)
        except Exception as e:
            return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [ ]:
scorepy = f"""import json
import pandas as pd
import joblib
from azureml.core.model import Model

def init():
    global model
    global selected_columns

    # Cargar el modelo registrado
    model_path = Model.get_model_path('{mname}')  # Asegúrate que este sea el nombre correcto
    model = joblib.load(model_path)

    # Cargar las columnas seleccionadas
    selected_columns_path = Model.get_model_path('selected_columns.json')
    with open(selected_columns_path, 'r') as f:
        selected_columns = json.load(f)

def run(raw_data):
    try:
        # Convertir el string JSON a DataFrame
        data_dict = json.loads(raw_data)
        data = pd.DataFrame.from_dict(data_dict["data"])

        # Filtrar columnas esperadas por el modelo
        data = data[selected_columns]

        # Predecir
        predictions = model.predict(data).tolist()

        return json.dumps({"predictions": predictions})
    except Exception as e:
        return json.dumps({"error": str(e)})

"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

### Step 2: Inference configuration

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Check if the service already exists
service_name = 'service'
if service_name in Webservice.list(ws):
    existing_service = Webservice(ws, service_name)
    existing_service.delete()  # Delete the existing service to avoid conflict

# Create the environment
virtual_env = Environment(".venv")
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas', 'joblib', 'azureml-defaults', 'scikit-learn']
)  # Ensure to coordinate with the model team for correct packages

# Configure inference and deployment
inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="score.py",
)
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)  # Ensure sufficient resources

# Deploy the model
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True,
)

C:\Users\rodri\AppData\Local\Temp\ipykernel_11748\1621776296.py:26: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [7]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: aeeecb87-125e-4e82-afc1-4648ff0596e5
More information can be found here: 
Error:
{
  "code": "EnvironmentBuildFailed",
  "statusCode": 400,
  "message": "Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: aeeecb87-125e-4e82-afc1-4648ff0596e5
More information can be found here: 
Error:
{
  "code": "EnvironmentBuildFailed",
  "statusCode": 400,
  "message": "Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: aeeecb87-125e-4e82-afc1-4648ff0596e5\nMore information can be found here: \nError:\n{\n  \"code\": \"EnvironmentBuildFailed\",\n  \"statusCode\": 400,\n  \"message\": \"Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information.\"\n}"
    }
}